In [9]:
import os, sys, numpy as np
import argparse
from time import time
from tqdm import tqdm
import tensorflow 
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data
import torchvision.transforms as transforms
from PIL import Image
import keras
from keras.layers import TimeDistributed as td
from keras.layers import Conv2D, Flatten, Dense, ZeroPadding2D, Activation
from keras.layers import MaxPooling2D, Dropout, BatchNormalization, Reshape
import os
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image, ImageOps
import random

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import random

In [19]:
dirlist=os.listdir('/content/drive/MyDrive/newtest')
xim=[]
yim=[]
random.shuffle(dirlist)
for image in dirlist:
  small=cv2.imread('/content/drive/MyDrive/newtest/'+image)
  print(small.shape)
  size=(128,128)
  img=cv2.resize(small,size,interpolation=cv2.INTER_CUBIC)
  img_normalized = cv2.normalize(img, None, -1.0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
  xim.append(img_normalized)
for image in dirlist:
  if (image[:2]=='14' or image[:2]=='15' ): #or image[:2]=='13' or image[:2]=='16'
      yim.append([0,1])
  else:
      yim.append([1,0])

(117, 117, 3)
(128, 128, 3)
(119, 119, 3)
(112, 112, 3)
(116, 116, 3)
(125, 125, 3)
(103, 103, 3)
(106, 106, 3)
(124, 124, 3)
(111, 111, 3)
(112, 112, 3)
(109, 109, 3)
(124, 124, 3)
(111, 111, 3)
(121, 121, 3)
(109, 109, 3)
(108, 108, 3)
(115, 115, 3)
(124, 124, 3)
(106, 106, 3)


In [20]:
print(yim)

[[0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [0, 1], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0]]


In [14]:
dirlist=os.listdir('/content/drive/MyDrive/test')
ximt=[]
yimt=[]
random.shuffle(dirlist)
for image in dirlist:
  small=cv2.imread('/content/drive/MyDrive/test/'+image)
  print(small.shape)
  size=(128,128)
  img=cv2.resize(small,size,interpolation=cv2.INTER_CUBIC)
  img_normalized = cv2.normalize(img, None, -1.0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
  ximt.append(img_normalized)
yimt=[[0,1],[0,1],[0,1],[0,1]]

(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)


In [15]:
import torch.nn as nn
from torchsummary import summary
import torch.nn.functional as F

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1) #128 x 128 x 32
        self.mp1=nn.MaxPool2d(2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1) #64 x 64 x 64
        self.mp2=nn.MaxPool2d(2, stride=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1) #32 x 32 x 128
        self.mp3=nn.MaxPool2d(2, stride=2)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1) #16 x 16 x 256
        self.mp4=nn.MaxPool2d(2, stride=2)
        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1) #8 x 8 x 512
        self.mp5=nn.MaxPool2d(2, stride=2)
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(512*4*4, 128)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 2)
        self.softmax = nn.Softmax()

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.mp1(x)
        x = F.relu(self.conv2(x))
        x = self.mp2(x)
        x = F.relu(self.conv3(x))
        x = self.mp3(x)
        x = F.relu(self.conv4(x))
        x = self.mp4(x)
        x = F.relu(self.conv5(x))
        x = self.mp5(x)
        x = self.flat(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.softmax(x)

        return x

disc= Discriminator()

In [16]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(disc.parameters(), lr=0.001)

In [ ]:
tensorx = torch.from_numpy(np.array(xim)).permute(0,3,1,2)
tensory = torch.from_numpy(np.array(yim))

for epoch in range(50):  # loop over the dataset multiple times
    for i in range (20):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = tensorx[i:i+1].float(),tensory[i:i+1].float()
        #print(inputs.shape)
        #print(labels.shape)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = disc(inputs)
        print (outputs,labels)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
    print(loss.item())

print('Finished Training')

In [22]:
tensorxt = torch.from_numpy(np.array(ximt)).permute(0,3,1,2)
tensoryt = torch.from_numpy(np.array(yimt))

for i in range (4):
    inputs, labels = tensorxt[i:i+1].float(),tensoryt[i:i+1].float()
    #print(inputs.shape)
    #print(labels.shape)
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = disc(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
print(loss.item())

print('Finished Testing')

<ipython-input-15-3040ffe721aa>:39: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


0.6931471824645996
Finished Testing
